In [11]:
import torch
import torch.utils.data as data
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
from torch.utils.data import Dataset
import torchvision as tv

import os,sys
import pandas as pd
import numpy as np
import pickle
from PIL import Image
# import spacy
# import scispacy
from tqdm import tqdm 
# from nltk.tokenize import RegexpTokenizer
# from transformers import BertTokenizer

In [2]:
data_dict = [
    {'img':'files/p19/p19017919/s50797181/8e5f8df3-4248d575-fd16f8c1-1c69065a-1ac199e9.dcm', 'txt':1},
    {'img':'files/p18/p18441078/s58044074/15e60dc7-6493889f-d2cb9dba-2c860709-f171bc98.dcm', 'txt':2},
    {'img':'files/p11/p11673861/s53517205/0491306a-d44bb1cd-36678cdf-ef64aa11-ee9897a2.dcm', 'txt':3},
    {'img':'files/p15/p15398519/s52897917/6f9899f3-652a0fe6-cb32f8b1-3afb0c85-5a3929f0.dcm', 'txt':4},
    {'img':'files/p13/p13118941/s54144739/67b68818-61703959-6321083a-ecf0ff32-addd5bc8.dcm', 'txt':5},
    {'img':'files/p13/p13660399/s52532898/2f2d7a89-161059ab-bf24aaa4-c15f7abb-ea408fee.dcm', 'txt':6},
    {'img':'files/p14/p14882082/s59150331/5009240f-7dc4006c-4a69d332-00051c01-9179f001.dcm', 'txt':7},
    {'img':'files/p17/p17198774/s54162995/2d7e35ed-6f80d557-740804a5-c1cdc6f0-5729e6a6.dcm', 'txt':8},
    {'img':'files/p17/p17199342/s59731194/b3e8c46c-99efe614-59b11837-a870ab41-8e30ceea_err.dcm', 'txt':9},
    {'img':'files/p18/p18446282/s54914372/32b63c6b-837955f2-62cd29e6-834d5132-6f9c67d8_err.dcm', 'txt':10},
    {'img':'files/p11/p11285576/s54979966/03b2e67c-70631ff8-685825fb-6c989456-621ca64d.dcm', 'txt':11},
    {'img':'files/p11/p11378535/s50374044/db9511e3-ee0359ab-489c3556-4a9b2277-c0bf0369.dcm', 'txt':12},
#     {'img':'files/p14/p14887088/s54257662/1bc85033-355accce-e8d0ed50-78188cd3-dac92e86.dcm', 'txt':13},
]


MAX_DIM = 64

class TestDataset(Dataset):
    def __init__(self, dataset):
        super().__init__()
        
        self.root = '/media/MyDataStor2/MIMIC-CXR/physionet.org/files/mimic-cxr-jpg/2.0.0'
        self.transform = tv.transforms.Compose([
                                        tv.transforms.RandomCrop(MAX_DIM),
                                        tv.transforms.ToTensor()])
        self.datadict = dataset

    def __len__(self):
        return len(self.datadict)

    def __getitem__(self, idx):
        
        image_path = os.path.join(self.root, self.datadict[idx]['img'].replace('dcm','jpg'))

        try:
            image = Image.open(image_path)
            if self.transform:
                image = self.transform(image)

        except Exception as ex:
            print(ex)
            print(image_path)
            return None
        
        text = self.datadict[idx]['txt']

        return image, text, os.path.basename(image_path)

In [3]:
def collate_fn_ignore_none(batch):
    batch = list(filter(lambda x: x is not None, batch))
    return torch.utils.data.dataloader.default_collate(batch)

In [4]:
testset = TestDataset(data_dict)
# print(len(testset))
testloader = torch.utils.data.DataLoader(testset, collate_fn=collate_fn_ignore_none, shuffle=True, batch_size=3, num_workers=2)

In [5]:
for image, text, pth in testloader:
    print(image.shape, text.shape, len(pth), sep='\n')
    for i in range(len(pth)):
        print('%02d - %s' % (text[i], pth[i]))

[Errno 2] No such file or directory: '/media/MyDataStor2/MIMIC-CXR/physionet.org/files/mimic-cxr-jpg/2.0.0/files/p17/p17199342/s59731194/b3e8c46c-99efe614-59b11837-a870ab41-8e30ceea_err.jpg'
/media/MyDataStor2/MIMIC-CXR/physionet.org/files/mimic-cxr-jpg/2.0.0/files/p17/p17199342/s59731194/b3e8c46c-99efe614-59b11837-a870ab41-8e30ceea_err.jpg
[Errno 2] No such file or directory: '/media/MyDataStor2/MIMIC-CXR/physionet.org/files/mimic-cxr-jpg/2.0.0/files/p18/p18446282/s54914372/32b63c6b-837955f2-62cd29e6-834d5132-6f9c67d8_err.jpg'
/media/MyDataStor2/MIMIC-CXR/physionet.org/files/mimic-cxr-jpg/2.0.0/files/p18/p18446282/s54914372/32b63c6b-837955f2-62cd29e6-834d5132-6f9c67d8_err.jpg
torch.Size([3, 1, 64, 64])
torch.Size([3])
3
08 - 2d7e35ed-6f80d557-740804a5-c1cdc6f0-5729e6a6.jpg
03 - 0491306a-d44bb1cd-36678cdf-ef64aa11-ee9897a2.jpg
06 - 2f2d7a89-161059ab-bf24aaa4-c15f7abb-ea408fee.jpg
torch.Size([2, 1, 64, 64])
torch.Size([2])
2
02 - 15e60dc7-6493889f-d2cb9dba-2c860709-f171bc98.jpg
01 - 8e5

In [31]:
pths = '/media/MyDataStor2/MIMIC-CXR/physionet.org/files/mimic-cxr-jpg/2.0.0/files/p11/p11285576/s54979966/03b2e67c-70631ff8-685825fb-6c989456-621ca64d.jpg'
img = Image.open(pths)
img.size

(3050, 697)

## Check dataloader

In [12]:
from dataset_mimic import build_dataset
from misc.config import Config

cfg  = Config()

In [13]:
data_set = build_dataset('train', cfg)
train_loader = torch.utils.data.DataLoader(
                data_set, batch_size=cfg.batch_size, 
                collate_fn=collate_fn_ignore_none, drop_last=True,
                shuffle=True, num_workers=cfg.num_workers)

In [14]:
cnt = 0
for image , caption, cap_mask, uid, cap_lens in train_loader:
    print(image.shape, caption.shape, cap_mask.shape, cap_lens.shape, len(uid))
    cnt += 1
    if cnt == 1:
        break

torch.Size([32, 1, 2048, 2048]) torch.Size([32, 257]) torch.Size([32, 257]) torch.Size([32]) 32
torch.Size([32, 1, 2048, 2048]) torch.Size([32, 257]) torch.Size([32, 257]) torch.Size([32]) 32


In [25]:
p = 1
print(cap_lens[p])
print(caption[p, cap_mask[p]])
print(' '.join([idx2word[x.item()] for x in caption[p, cap_mask[p]]]))

tensor(178)
tensor([   1,  509,  450,  599,   73,   40,   22,   74,  239,  257,   43,  120,
          95,   22,  121,  174,  397,  462,  412,  487,   42,  427,  310,   22,
         463,   91,  676,  412,  487,  107,   22,   35,  414,   91,   79,  707,
         399,  487,  107,   22,  270,  871, 1189, 1190,   22,   79,   74,  412,
         124,  125,  424,  425,   43,  124,   91,  591,  755,  590,  107,   22,
         514, 1471,   91,   29,   74,  412,    5, 3403, 3404,   43,  281,  591,
         733,   45,  107,   22,   29,  214,  316,   79,   99,   12,    5,  153,
         351,   79,  270,  172,  341,    5,  156,  434,   79,  365, 6412,    5,
         268,  153,   22,   29,   30,    5,   57,   43,    4,    5,    6,  126,
          29,    9,   10,   11,   29,   12,   22,   82,   43,   51,   53,   45,
         268,  153,   91,   29, 1393, 1388,   43,  480,   45,  153,  107,  446,
          29,   64, 1074,   77,    5,  268,  153,   42,  156,  351,   79,  270,
         172,  341,   42,   

In [17]:
data_dir = '/media/MyDataStor2/MIMIC-CXR/'
with open(os.path.join(data_dir,'lm_reports/mimic_dataset_mit_normalized.pkl'),'rb') as f:
    datasets = pickle.load(f)

In [19]:
word2idx = datasets['word2idx']
idx2word = datasets['idx2word']